In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix

import numpy as np

path = "../Data/"
words = []


def train_to_sentence_form(path):
    # reads the file to a format of docs[doc[[sentence1],[sentence2]],...]
    docs=[]
    num_of_sentences = []
    with open(path) as f:
        line = f.readline()
        while line:
            line = line.rstrip('\n').split(" ")
            #print(line)
            num = line.pop(0)
            #print(num)
            number = int(num[1:-1])
            num_of_sentences.append(number)
            len_of_sentence = []

            for i,k in enumerate(line):
                if k.startswith('<'):
                    k=k[1:-1]
                    len_of_sentence.append(int(k))
                    line.pop(i)
            doc_sentence=[]       
            sentence=[]
            for j in len_of_sentence:
                sentence = line[:j]
                line=line[j:]
                docs.append( sentence) 
            line = f.readline()
    print(len(docs))
    return docs,num_of_sentences     
    #docs is a list of lists where each list is a different sentence





vocabs = open(path+"vocabs.txt").readlines()
for vocab in vocabs:
    word, _ = vocab.split(', ')
    words.append(word)

train_label = [i.strip() for i in open(path+"train-label.dat").readlines()]
train_label = np.array(train_label)

#path="../Data/"
docs,num_sentences = train_to_sentence_form(path+"train-data.dat")

#train_data = convert_to_word(path + "/train-data.dat")
data = []
for i in docs:
    final_L = ''
    for l in i:
        final_L = final_L + ' ' + words[int(l)]
    data.append(final_L[1:])
print(data[:12])
print(len(data))       
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(data)
print(X_train_counts.size)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

149925
['rubi rail helper demo more info auto complet', 'see new helper action', 'null length substr locat', 'exec messag messag pleas edit remov follow word content', 'roll stone com news song previou next page', 'good vibrat beach boy', 'smell teen spirit nirvana', 'want hold hand beatl', 'hound dog elvi', 'god know beach boy', 'walk line johnni cash', 'heaven led zeppelin']
149925
965852


In [2]:
def multilabel_confusion_matrix(predict,path,clustnum=0):
    test_label = [i.strip().split() for i in open(path + "test-label.dat").readlines()]

    for i in range(len(test_label)):
        item = test_label[i]
        for j in range(len(item)):
            item[j] = int(item[j])

        test_label[i] = item

    labels = []

    for item in open(path + "labels.txt").readlines():
        item = item.replace('\n', '')
        label_name, _ = item.split(', ')
        labels.append(label_name)

    y_true = np.array(test_label)
    y_pred = np.array(predict)

    conf_mat_dict = {}

    for label_col in range(len(labels)):
        y_true_label = y_true[:, label_col]
        y_pred_label = y_pred[:, label_col]
        conf_mat_dict[labels[label_col]] = confusion_matrix(y_pred=y_pred_label, y_true=y_true_label)
    f = open("confmatrix"+str(clustnum)+".txt", "w")
    f.close()
    for label, matrix in conf_mat_dict.items():
        print("Confusion matrix for label {}:".format(label))
        f = open("confmatrix"+str(clustnum)+".txt", "a")
        print(matrix)
        f.write("Confusion matrix for label {}:".format(label))
        np.savetxt(f,matrix)
        f.close()
    return

In [3]:
#from pandas import costcla
!pip install costcla
from costcla.metrics import cost_loss
from costcla.models import CostSensitiveLogisticRegression

from sklearn.linear_model import LogisticRegression

def calculate_cost_matrix(y_list):
    y_list_len = len(y_list)
    percentage_0 = np.count_nonzero(y_list == 0) / y_list_len
    percentage_1 = np.count_nonzero(y_list == 1) / y_list_len

    cost_matrix = []
    for i in range(len(y_list)):
        cost_matrix.append([1 / (percentage_0 * y_list_len), 1 / (percentage_1 * y_list_len), 0, 0])

    return np.array(cost_matrix)


C:\Users\Peris\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Peris\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [4]:
import pandas as pd

test_data,test_sentences = train_to_sentence_form(path+"test-data.dat")
test_label = [i.strip() for i in open(path+"test-label.dat").readlines()]
test_label = np.array(test_label)
train_label = [i.strip() for i in open(path+"train-label.dat").readlines()]
train_label = np.array(train_label)

df = pd.read_csv("../class-imbalance/train_kmeans_.csv") 
df_test = pd.read_csv("../class-imbalance/test_kmeans_.csv")
#train_label = pd.DataFrame(data = train_label)
labels_models=[]
for i in range (0,20 ):
    tr=[]
    for j in range (0,len(train_label)):
        tr.append(int(train_label[j][2*i]))
    labels_models.append(tr)
    
train_labels = pd.DataFrame(labels_models)
train_labels = train_labels.transpose()
#print(train_labels[:][2])
labels__models=[]
for i in range (0,20 ):
    ts=[]
    for j in range (0,len(test_label)):
        ts.append(int(test_label[j][2*i]))
    labels__models.append(ts)
    
test_labels = pd.DataFrame(labels__models)
test_labels = test_labels.transpose()


73363


In [5]:
cost_mat_train = calculate_cost_matrix(train_labels)
f = CostSensitiveLogisticRegression()
cost_mat_test = calculate_cost_matrix(test_labels)
pred = pd.DataFrame()
print(df)
#drop 1st column because of an error in 1st column
df = df.drop(df.columns[0],axis=1)
df_test = df_test.drop(df_test.columns[0],axis=1)
print(df)

orderlist = [2,7,4,0,8,6,5,12,10,9,17,11,15,14,3,13,16,1,18,19]
for l in orderlist:
    print(df.shape)
    f = CostSensitiveLogisticRegression()
    f.fit(df, train_labels[:][l],cost_mat_train)
    df['Label'+str(l)] = train_labels[:][l]
    df_test['Label'+str(l)] = f.predict(df_test) 
    pred['Label'+str(l)] = df_test['Label'+str(l)]

predorder = ['Label'+str(i) for i in range(20)]
pred = pred[predorder]
cost_loss(df_test, pred, cost_mat_test)

      Unnamed: 0  Cluster 0  Cluster 1  Cluster 2  Cluster 3  Cluster 4  \
0              0        0.0        0.0        0.0        1.0        0.0   
1              1        0.0        2.0       24.0        0.0        0.0   
2              2        0.0        1.0       17.0        1.0        0.0   
3              3        0.0        0.0        1.0        0.0        0.0   
4              4        0.0        0.0        1.0        2.0        0.0   
...          ...        ...        ...        ...        ...        ...   
8246        8246        0.0        0.0        4.0        0.0        0.0   
8247        8247        0.0        2.0       23.0        0.0        0.0   
8248        8248        0.0        0.0       15.0        0.0        0.0   
8249        8249        1.0        0.0        4.0        0.0        0.0   
8250        8250        0.0        0.0        1.0        0.0        0.0   

      Cluster 5  Cluster 6  Cluster 7  Cluster 8  ...  Cluster 10  Cluster 11  \
0           0.0   

ValueError: bad input shape (3983, 40)

In [14]:
print(df_test)
print(pred)
print(cost_mat_test.shape)
for i in range(20):
    print(cost_loss(df_test.iloc[:,i], pred.iloc[:,i], cost_mat_test))
    
predict=[]
for p in range(3983):
    #print(p)
    result = []
    for r in range(20):
        x = orderlist.index(r)
        result.append(pred.iloc[p,x])
    predict.append(result)
multilabel_confusion_matrix(predict,path)

      Cluster 0  Cluster 1  Cluster 2  Cluster 3  Cluster 4  Cluster 5  \
0           0.0        1.0        9.0        0.0        1.0        0.0   
1           0.0        0.0       22.0        2.0        0.0        0.0   
2           2.0        0.0       21.0        1.0        0.0        0.0   
3           2.0        0.0        9.0        0.0        0.0        0.0   
4           0.0        0.0        1.0        0.0        0.0        1.0   
...         ...        ...        ...        ...        ...        ...   
3978        1.0        0.0        6.0        0.0        0.0        0.0   
3979        0.0        2.0       17.0        1.0        5.0        0.0   
3980        3.0        0.0        9.0        3.0        0.0        0.0   
3981        0.0        0.0        1.0        0.0        0.0        1.0   
3982        0.0        0.0        1.0        0.0        0.0        0.0   

      Cluster 6  Cluster 7  Cluster 8  Cluster 9  Cluster 10  Cluster 11  \
0           2.0        2.0        0